In [ ]:
%pylab inline
%load_ext version_information

In [ ]:
from __future__ import division, print_function
from IPython.core.display import Image
from deltasigma import *
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [ ]:
np.set_printoptions(suppress=True, precision=3)

Delta Sigma modulator synthesis - example #1
============================================

Demonstration of the synthesis of an example delta sigma modulator,  as done in the `MATLAB Delta Sigma Toolbox`, employing its Python port `deltasigma`.

 * The **Noise Transfer Function** (NTF) is synthesized for a **5th-order**, **low-pass** modulator, having the following characteristics:

     * A relatively low Over Sampling Ratio (OSR), equal to `32`, 
     * A 1-bit quantizer. 
     * Maximum NTF gain equal to 1.5.

 * For this example modulator, we select to implement the CRFB topology -- see below, for order being odd -- with a single feed-in, ie $b_n = 0\ \forall n > 1$ .

In [ ]:
Image(url='http://python-deltasigma.readthedocs.org/en/latest/_images/CRFB.png', retina=True)

## Set up the parameters

In [ ]:
order = 5
osr = 32
nlev = 2
f0 = 0.
Hinf = 1.5
form = 'CRFB'

## Synthesize the NTF

In [ ]:
ntf = synthesizeNTF(order, osr, 2, Hinf, f0)            # Optimized zero placement
print("Synthesized a %d-order NTF, with roots:\n" % order)
print(" Zeros:\t\t\t Poles:")
for z, p in zip(ntf[0], ntf[1]):
    print("(%f, %fj)\t(%f, %fj)" % (np.real(z), np.imag(z), np.real(p), np.imag(p)))
print("")
print("The NTF transfer function has the following expression:\n")
print(pretty_lti(ntf))
print("")

### Graphical inspection of the synthesized singularities

In [ ]:
plotPZ(ntf, showlist=True)

## Realize the NTF with the CRFB topology and create the ABCD representation

In [ ]:
a, g, b, c = realizeNTF(ntf, form)
b = np.hstack(( # Use a single feed-in for the input
               np.atleast_1d(b[0]),
               np.zeros((b.shape[0] - 1, ))
             ))
ABCD = stuffABCD(a, g, b, c, form)
print("ABCD Matrix:")
print(ABCD)

In [ ]:
DocumentNTF(ABCD, osr, f0)
f = gcf()
f.set_size_inches((15, 6))

## Typical spectral characteristics

In [ ]:
figure(figsize=(15,8))
PlotExampleSpectrum(ntf, M=1, osr=osr, f0=f0)

## Signal to Noise Ratio analysis

In [ ]:
snr, amp = simulateSNR(ntf, osr, None, f0, nlev)

In [ ]:
figure(figsize=(15,8))
if nlev == 2:
    snr_pred, amp_pred, k0, k1, se = predictSNR(ntf, osr)
    plot(amp_pred, snr_pred, '-', label='predicted')
plot(amp, snr,'o-.g', label='simulated')
xlabel('Input Level (dBFS)')
ylabel('SQNR (dB)')
peak_snr, peak_amp = peakSNR(snr, amp)
msg = 'peak SQNR = %4.1fdB  \n@ amp = %4.1fdB  ' % (peak_snr, peak_amp)
text(peak_amp-10,peak_snr,msg, horizontalalignment='right', verticalalignment='center');
msg = 'OSR = %d ' % osr
text(-2, 5, msg, horizontalalignment='right');
figureMagic([-100, 0], 10, None, [0, 100], 10, None, [12, 6], 'Time-Domain Simulations')
legend(loc=2);

## Dynamic range scaling

Keep the integrator outputs within their working range -- to avoid waveform clipping and instabilities.

The ABCD matrix is scaled opportunely, then the effectiveness of the dynamic range scaling is verified and the state maxima are plotted for different amplitudes.

In [ ]:
print('Doing dynamic range scaling... ')
ABCD0 = ABCD.copy()
ABCD, umax, S = scaleABCD(ABCD0, nlev, f0)
print('Done.')
print("Maximum input magnitude: %.3f" % umax)

In [ ]:
print('Verifying dynamic range scaling... ')
u = np.linspace(0, 0.95*umax, 30)
N = 1e4
N0 = 50
test_tone = np.cos(2*np.pi*f0*np.arange(N))
test_tone[:N0] = test_tone[:N0]*(0.5 - 0.5*np.cos(2*np.pi/N0*np.arange(N0)))
maxima = np.zeros((order, u.shape[0]))
for i in np.arange(u.shape[0]):
    ui = u[i]
    v, xn, xmax, y = simulateDSM(ui*test_tone, ABCD, nlev)
    maxima[:, i] = xmax[:, 0]
    if (xmax > 1e2).any(): 
        print('Warning, umax from scaleABCD was too high.')
        umax = ui
        u = u[:i]
        maxima = maxima[:, :i]
        break
print('Done.')
print("Maximum DC input level: %.3f" % umax)

In [ ]:
colors = get_cmap('jet')(np.linspace(0, 1.0, order))
for i in range(order):
    plot(u,maxima[i,:], 'o-', color=colors[i], label='State %d' % (i+1))
grid(True)
#text(umax/2, 0.05, 'DC input', horizontalalignment='center', verticalalignment='center')
figureMagic([0, umax], None, None, [0, 1] , 0.1, 2, [12, 6], 'State Maxima')
xlabel('DC input')
ylabel('Maxima')
legend(loc='best');

## Conclusions
### Scaled coefficients

In [ ]:
a, g, b, c = mapABCD(ABCD, form)

In [ ]:
adc = {
       'order':order,
       'osr':osr,
       'nlev':nlev,
       'f0':f0,
       'ntf':ntf,
       'ABCD':ABCD,
       'umax':umax,
       'peak_snr':peak_snr,
       'form':form,
       'coefficients':{
                       'a':a,
                       'g':g,
                       'b':b,
                       'c':c
                      }
      }

### Final ADC coefficients

In [ ]:
t_sz = max([len(adc['coefficients']['a']), len(adc['coefficients']['g']), \
            len(adc['coefficients']['b']), len(adc['coefficients']['c'])])
t_nan = [np.nan for x in range(t_sz)]
t = {'Coefficients' : ['#1', '#2', '#3', '#4', '#5', '#6'],
     'DAC feedback: a(n)' : (adc['coefficients']['a'].tolist()+t_nan)[:t_sz],
     'Resonator feedback: g(n)' : (adc['coefficients']['g'].tolist()+t_nan)[:t_sz],
     'Feed-in: b(n)' : (adc['coefficients']['b'].tolist()+t_nan)[:t_sz],
     'Interstage: c(n)' : (adc['coefficients']['c'].tolist()+t_nan)[:t_sz]}
pd.DataFrame(t)

### System version information

In [ ]:
%version_information numpy, scipy, matplotlib, deltasigma, cython